<a href="https://colab.research.google.com/github/LeahN67/Musical_Genre_Classification/blob/master/Spectrogram_Code_4_beats_optimized_98__accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Here is the deep learning code for respirosa

In [ ]:
# make your Google Drive accessible
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount=True)


In [ ]:
from fastai.vision import *


In [ ]:
# Here we create a function that creates spectograms using the library librosa

import os
import glob
import shutil
import random

from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import librosa
import librosa.display
import numpy as np



def create_fold_spectrograms(fold):
    spectrogram_path = Path('/content/drive/My Drive/audio/spectrogram/')
    audio_path = Path('/content/drive/My Drive/audio/')
    print(f'Processing fold {fold}')
    os.mkdir(spectrogram_path/fold)
    for audio_file in list(Path(audio_path/f'fold{fold}').glob('*.wav')):
        samples, sample_rate = librosa.load(audio_file)
        fig = plt.figure(figsize=[0.72,0.72])
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)
        filename  = spectrogram_path/fold/Path(audio_file).name.replace('.wav','.png')
        S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
        plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
        plt.close('all') (edited) 

 #This call here creates spectograms that are saved in the different folds

for i in range(1, 3):
    create_fold_spectrograms(str(i)) (edited)




In [ ]:
# # Creating Train / Val / Test folders (One time use)
root_dir = '/content/gdrive/My Drive/4beats'
classes_dir = ['/60bpm', '/70bpm', '/75bpm', '/76bpm','/77bpm','/78bpm','/79bpm','/80bpm','/82bpm','/83bpm','/84bpm','/85bpm','/86bpm','/89bpm','/90bpm']

val_ratio = 0.20

for cls in classes_dir:
    os.makedirs(root_dir +'/train' + cls)
    os.makedirs(root_dir +'/val' + cls)


    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio ))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]

    print('Total images: '+ str(cls), len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'/train' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_dir +'/val' + cls)

In [ ]:
path = '/content/gdrive/My Drive/4beats/'
img_size = 224
tfms = get_transforms(do_flip=False)
bs = 64
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, train='train', valid='val', size=img_size, bs = bs) 

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy)


In [ ]:
learn.fit(10)



In [ ]:
learn.save('stage-1')
learn.load('stage-1')
learn.unfreeze()
learn.fit_one_cycle(12, max_lr=slice(1e-5, 1e-4))


In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.save('stage-2')
learn.load('stage-2')
learn.unfreeze()
learn.fit_one_cycle(12, max_lr=slice(1e-5, 1e-4))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_top_losses(9, figsize=(12,12) )


In [ ]:
interp.plot_confusion_matrix()


In [ ]:
interp.most_confused(min_val=1)
